In [1]:
import pandas as pd
import numpy as np
from pgmpy.models import MarkovNetwork
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.inference import VariableElimination, BeliefPropagation

c:\Users\ddsdi\.virtualenvs\lightGCN-gUtlu06Z\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = MarkovNetwork()

edges = [
    ('X1', 'X2'), ('X2', 'X3'), ('X3', 'X4'), ('X1', 'X4'),
    ('X1', 'Y1'), ('X2', 'Y2'), ('X3', 'Y3'), ('X4', 'Y4')
]
model.add_edges_from(edges)

In [3]:
# (0 --> B, 1 --> W)
states = [0, 1]

# Add potential functions phi(Xi, Xj)
potential_values = np.array([[2, 0.5], [0.5, 2]])

# Add factors (potential functions) for the Xi-Xj connections
for edge in [('X1', 'X2'), ('X2', 'X3'), ('X3', 'X4'), ('X1', 'X4')]:
    factor = DiscreteFactor([edge[0], edge[1]], [2, 2], potential_values.flatten())
    model.add_factors(factor)

In [4]:
# Add lambda(Xi, Yi) potential functions
lambda_values = np.array([[4, 0.5], [0.5, 4]])

for edge in [('X1', 'Y1'), ('X2', 'Y2'), ('X3', 'Y3'), ('X4', 'Y4')]:
    factor = DiscreteFactor([edge[0], edge[1]], [2, 2], lambda_values.flatten())
    model.add_factors(factor)

In [5]:
evidence = {
    'Y1': 0,  # B
    'Y2': 1,  # W
    'Y3': 0,  # B
    'Y4': 1   # W
}

In [6]:
belief_propagation = BeliefPropagation(model)
print(belief_propagation.query(variables=['X1', 'X2'], evidence=evidence))

+-------+-------+--------------+
| X1    | X2    |   phi(X1,X2) |
+=======+=======+==============+
| X1(0) | X2(0) |       0.3465 |
+-------+-------+--------------+
| X1(0) | X2(1) |       0.2897 |
+-------+-------+--------------+
| X1(1) | X2(0) |       0.0173 |
+-------+-------+--------------+
| X1(1) | X2(1) |       0.3465 |
+-------+-------+--------------+


In [7]:
0.0173/0.2897

0.0597169485674836

### The below code segment is the part equivalent to the solution given in the assignment.

In [8]:
infer = VariableElimination(model)
print(infer.query(variables=['X1', 'X2'], evidence=evidence))

+-------+-------+--------------+
| X1    | X2    |   phi(X1,X2) |
+=======+=======+==============+
| X1(0) | X2(0) |     100.5000 |
+-------+-------+--------------+
| X1(0) | X2(1) |      84.0000 |
+-------+-------+--------------+
| X1(1) | X2(0) |       5.0039 |
+-------+-------+--------------+
| X1(1) | X2(1) |     100.5000 |
+-------+-------+--------------+


In [9]:
5.0039/84

0.05957023809523809